In [1]:
#Mouting drive for getting access to MyDrive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#Installing pyspark
!pip install pyspark

     |████████████████████████████████| 204.2MB 72kB/s 
     |████████████████████████████████| 204kB 51.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=08b4874a2aaa8ed2ac63f06b6bf99be67436cb8ee8cac964231103a87e92b345
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [4]:
#Importing required libraries
import pyspark

In [6]:
from pyspark import SparkContext
#to stop the sparkcontext if its already running
#sc.stop()
#setting sparkcontext locally
sc = SparkContext("local", "rest_app")

In [7]:
#creating sqlcontext for accessing SparkSQL
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [8]:
#defining schema for the data as required within dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, StringType
schema = StructType([\
        StructField("user_id_key", IntegerType()),\
        StructField("business_id_key", IntegerType()),\
        StructField("stars", FloatType()),\
        StructField("business_name", StringType()),\
    ])

In [17]:
#creating sparksql dataframe with set schema from ratings.csv file
spark_df = sqlContext.read.csv("drive/My Drive/CMPE-256-Project/MonYelpMergedData(final_processed).csv", header=True, sep=",", schema=schema)

In [18]:
#displaying the head part to check whether dataframe is created properly or not
spark_df.head()

Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous")

In [19]:
#displaying first 20 rows from dataframe
spark_df.head(20)

[Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=1, business_id_key=1, stars=5.0, business_name='Copper Branch'),
 Row(user_id_key=2, business_id_key=2, stars=3.0, business_name='Stash Café'),
 Row(user_id_key=3, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=4, business_id_key=3, stars=4.0, business_name='La Société Montréal'),
 Row(user_id_key=5, business_id_key=4, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=6, business_id_key=5, stars=3.0, business_name='La Chronique'),
 Row(user_id_key=6, business_id_key=6, stars=4.0, business_name='Toi Moi & Café'),
 Row(user_id_key=7, business_id_key=7, stars=5.0, business_name='Saint-Laurent Warehouse'),
 Row(user_id_key=8, business_id_key=8, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=8, business_id_key=9, stars=4.0, business_name='Cacao 70'),
 Row(user_id_key=8, business_id_key=10, stars=3.0, business_name='Pasta Casareccia'),
 Row(user_id_k

In [20]:
#checkig the datatype of each attribute
spark_df.dtypes

[('user_id_key', 'int'),
 ('business_id_key', 'int'),
 ('stars', 'float'),
 ('business_name', 'string')]

In [21]:
spark_df.count()

3437

In [22]:
##split the ratings into training and testing data
training, testing = spark_df.randomSplit([0.9, 0.1], seed=12345)

In [49]:
training.dtypes

[('user_id_key', 'int'),
 ('business_id_key', 'int'),
 ('stars', 'float'),
 ('business_name', 'string')]

In [50]:
training.count()

3129

In [51]:
testing.count()

308

In [52]:
training.head(3)

[Row(user_id_key=0, business_id_key=0, stars=5.0, business_name="Dunn's Famous"),
 Row(user_id_key=1, business_id_key=1, stars=5.0, business_name='Copper Branch'),
 Row(user_id_key=2, business_id_key=2, stars=3.0, business_name='Stash Café')]

In [ ]:
#saving training data to csv file in training.csv folder ,using repartition() to save all the data samples in single csv file
csv_location = 'drive/My Drive/CMPE-256-Project/mon_rest_training.csv'
training.repartition(1).write.csv(path=csv_location, mode="append", header="true")

In [ ]:
#saving testing data to csv file in testing.csv folder ,using repartition() to save all the data samples in single csv file
csv_location = 'drive/My Drive/CMPE-256-Project/mon_rest_testing.csv'
testing.repartition(1).write.csv(path=csv_location, mode="append", header="true")

In [9]:
#Importing the required libraries
import numpy as np 
import pandas as pd

In [ ]:
##Visualizing data for further analysis

In [53]:
training_df = pd.read_csv("drive/My Drive/CMPE-256-Project/mon_rest_training.csv")

In [11]:
training_df.head(20)

,user_id_key,business_id_key,stars,business_name
0,0,0,5.0,Dunn's Famous
1,1,1,5.0,Copper Branch
2,2,2,3.0,Stash Café
3,3,0,5.0,Dunn's Famous
4,4,3,4.0,La Société Montréal
5,5,4,4.0,Cacao 70
6,6,5,3.0,La Chronique
7,6,6,4.0,Toi Moi & Café
8,7,7,5.0,Saint-Laurent Warehouse
9,8,8,4.0,Cacao 70


In [12]:
testing_df = pd.read_csv("drive/My Drive/CMPE-256-Project/mon_rest_testing.csv")

In [13]:
testing_df.head(20)

,user_id_key,business_id_key,stars,business_name
0,14,18,4.0,Wendy's
1,15,19,4.0,Chez Ma Tante
2,21,9,4.0,Cacao 70
3,28,27,4.0,Pigeon
4,30,12,1.0,Les 3 Brasseurs
5,49,26,4.0,Les 3 Brasseurs
6,50,9,4.0,Cacao 70
7,53,12,3.0,Les 3 Brasseurs
8,63,26,2.0,Les 3 Brasseurs
9,65,9,3.0,Cacao 70


In [ ]:
##using Spark ML libraries for collaborative filtering

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

##User Based

In [23]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_key", itemCol="business_id_key", ratingCol="stars",
          coldStartStrategy="drop")
model = als.fit(training)

In [24]:
#creating testing dataframe with only userid and businessid as we need to predict ratings
testing_data=testing['user_id_key','business_id_key']

In [25]:
testing_data.head(5)

[Row(user_id_key=14, business_id_key=18),
 Row(user_id_key=15, business_id_key=19),
 Row(user_id_key=21, business_id_key=9),
 Row(user_id_key=28, business_id_key=27),
 Row(user_id_key=30, business_id_key=12)]

In [26]:
#applying trained model on testing data and evaluating predicted ratings
predictions = model.transform(testing_data)

In [27]:
testing_data.head(5)

[Row(user_id_key=14, business_id_key=18),
 Row(user_id_key=15, business_id_key=19),
 Row(user_id_key=21, business_id_key=9),
 Row(user_id_key=28, business_id_key=27),
 Row(user_id_key=30, business_id_key=12)]

In [28]:
predictions.take(5)

[Row(user_id_key=1372, business_id_key=26, prediction=-0.4380345344543457),
 Row(user_id_key=855, business_id_key=26, prediction=5.051612854003906),
 Row(user_id_key=386, business_id_key=26, prediction=-2.3015551567077637),
 Row(user_id_key=49, business_id_key=26, prediction=3.4651920795440674),
 Row(user_id_key=314, business_id_key=26, prediction=1.1058948040008545)]

In [29]:
##create dataframe with original ratings for testing data
actual_test_ratings=testing['user_id_key','business_id_key','stars']

In [30]:
actual_test_ratings.head(5)

[Row(user_id_key=14, business_id_key=18, stars=4.0),
 Row(user_id_key=15, business_id_key=19, stars=4.0),
 Row(user_id_key=21, business_id_key=9, stars=4.0),
 Row(user_id_key=28, business_id_key=27, stars=4.0),
 Row(user_id_key=30, business_id_key=12, stars=1.0)]

In [31]:
##Set condition to join both dataframes the one with predicted ratings and one with actual ratings for test data
condi=[predictions.user_id_key == actual_test_ratings.user_id_key, predictions.business_id_key == actual_test_ratings.business_id_key]

In [32]:
##creating dataframe by performing inner join 
final_rating_testData = predictions.join(actual_test_ratings,condi,how='inner')

In [33]:
#visualizing final dataframe with predicted and actual ratings for test data
final_rating_testData.head(5)

[Row(user_id_key=1372, business_id_key=26, prediction=-0.4380345344543457, user_id_key=1372, business_id_key=26, stars=4.0),
 Row(user_id_key=855, business_id_key=26, prediction=5.051612854003906, user_id_key=855, business_id_key=26, stars=5.0),
 Row(user_id_key=386, business_id_key=26, prediction=-2.3015551567077637, user_id_key=386, business_id_key=26, stars=4.0),
 Row(user_id_key=49, business_id_key=26, prediction=3.4651920795440674, user_id_key=49, business_id_key=26, stars=4.0),
 Row(user_id_key=314, business_id_key=26, prediction=1.1058948040008545, user_id_key=314, business_id_key=26, stars=4.0)]

In [35]:
##calculating rmse values using "rating" and "prediction" columns from final_rating_testData frame

evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars",
                                predictionCol="prediction")
rmse = evaluator.evaluate(final_rating_testData)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 4.000175061723747


The RMSE value for testing data is 4.00

In [47]:
##saving predicted ratings in prediction.csv file for future reference
#using repartition() to save all the data samples in single csv file

csv_location = 'drive/My Drive/CMPE-256-Project/predictions_userALS.csv'
predictions.repartition(1).write.csv(path=csv_location, mode="append", header="true")

##Item based

In [36]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als1 = ALS(maxIter=5, regParam=0.01, userCol="business_id_key", itemCol="user_id_key", ratingCol="stars",
          coldStartStrategy="drop")
model1 = als1.fit(training)

In [37]:
#creating testing dataframe with only userid and businessid as we need to predict ratings
testing_data1=testing['business_id_key','user_id_key']

In [38]:
testing_data1.head(4)

[Row(business_id_key=18, user_id_key=14),
 Row(business_id_key=19, user_id_key=15),
 Row(business_id_key=9, user_id_key=21),
 Row(business_id_key=27, user_id_key=28)]

In [39]:
#applying trained model on testing data and evaluating predicted ratings
predictions1 = model1.transform(testing_data1)

In [40]:
predictions1.take(5)

[Row(business_id_key=9, user_id_key=65, prediction=3.0026960372924805),
 Row(business_id_key=27, user_id_key=28, prediction=0.6205934286117554),
 Row(business_id_key=26, user_id_key=1372, prediction=0.0828561782836914),
 Row(business_id_key=2, user_id_key=384, prediction=-0.8256113529205322),
 Row(business_id_key=51, user_id_key=860, prediction=0.6645869016647339)]

In [41]:
##create dataframe with original ratings for testing data
actual_test_ratings1=testing['business_id_key','user_id_key','stars']

In [42]:
actual_test_ratings1.head(5)

[Row(business_id_key=18, user_id_key=14, stars=4.0),
 Row(business_id_key=19, user_id_key=15, stars=4.0),
 Row(business_id_key=9, user_id_key=21, stars=4.0),
 Row(business_id_key=27, user_id_key=28, stars=4.0),
 Row(business_id_key=12, user_id_key=30, stars=1.0)]

In [43]:
##Set condition to join both dataframes the one with predicted ratings and one with actual ratings for test data
condi1=[predictions1.business_id_key == actual_test_ratings1.business_id_key, predictions1.user_id_key == actual_test_ratings1.user_id_key]

In [44]:
##creating dataframe by performing inner join 
final_rating_testData1 = predictions1.join(actual_test_ratings1,condi,how='inner')

In [45]:
#visualizing final dataframe with predicted and actual ratings for test data
final_rating_testData1.head(5)

[Row(business_id_key=9, user_id_key=65, prediction=3.0026960372924805, business_id_key=9, user_id_key=65, stars=3.0),
 Row(business_id_key=27, user_id_key=28, prediction=0.6205934286117554, business_id_key=27, user_id_key=28, stars=4.0),
 Row(business_id_key=26, user_id_key=1372, prediction=0.0828561782836914, business_id_key=26, user_id_key=1372, stars=4.0),
 Row(business_id_key=2, user_id_key=384, prediction=-0.8256113529205322, business_id_key=2, user_id_key=384, stars=5.0),
 Row(business_id_key=51, user_id_key=860, prediction=0.6645869016647339, business_id_key=51, user_id_key=860, stars=4.0)]

In [46]:
##calculating rmse values using "rating" and "prediction" columns from final_rating_testData1 frame

evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="stars",
                                predictionCol="prediction")
rmse1 = evaluator.evaluate(final_rating_testData1)
print("Root-mean-square error = " + str(rmse1))

Root-mean-square error = 3.7878340019817305


The RMSE obtained for item based ALS recommendation is 3.78 which is better than user based ALS.

In [48]:
##saving predicted ratings in prediction.csv file for future reference
#using repartition() to save all the data samples in single csv file

csv_location = 'drive/My Drive/CMPE-256-Project/predictions_itemALS.csv'
predictions1.repartition(1).write.csv(path=csv_location, mode="append", header="true")